This file is the main file where we define the problem data, initiate model and train the model

1. First we import all the relevant classes and libraries

In [14]:
# from physicsinformed import InversePhysicsInformedBarModel # py file which has custom class defined
import torch
import torch.nn as nn
import plotly.graph_objects as go # for plotting the graph

Custom function needed to generate training and testing data

In [15]:
def generate_grid_1d(length, samples=20, initial_coordinate=0.0):
    """Generate an evenly space grid of a given length and a given number of samples."""

    # Generate the grid
    x = torch.linspace(initial_coordinate, initial_coordinate + length, samples, requires_grad=True)

    # Reshape on a column tensor and return
    return x.view(samples, 1)

2. Analytical 'u' is given

In [16]:
L = 1
x = generate_grid_1d(L, 20)
u_analytic = lambda x: torch.sin(2*torch.pi*x)
u = u_analytic(x)

In [17]:
# xu = torch.cat([x, u], 1)
# # xu.shape

Analytical inverse solution for validation

In [18]:
EA_analytic = lambda x: x**3 - x**2 + 1

3. Problem data are defined. Here the known data like boundary condition and loading conditions are defined

In [19]:
distLoad = lambda x: -2*(3*x**2 - 2*x)*torch.pi*torch.cos(2*torch.pi*x) + 4*(x**3 - x**2 + 1)*torch.pi**2*torch.sin(2*torch.pi*x)

4. Next we generate the neural network model using the imported class InversePhysicsInformedBarModel

In [20]:
# pinnModel = InversePhysicsInformedBarModel(x, u, L, distLoad) # custom class defined in physicsinformed.py file

In [21]:
import torch
import numpy as np

class PhysicsInformedBarModel:
    """A class used for the definition of Physics Informed Models for one dimensional bars."""

    def __init__(self,x, u, L, dist_load):
        # Construct a PhysicsInformedBar model
        # Initialize required variables for the class
        self.x = x
        self.u = u
        self.L = L
        self.dist_load = dist_load

    def costFunction(self, x, u, EA_pred,model):
        # Compute the cost function
        '''
        This function takes input x and model predicted output u_pred to compute loss
        Params:
            x - spatial value
            u_pred - NN predicted displacement value
            differential_equation_loss - calculated PDE residual loss
            boundary_condition_loss - calculated boundary loss
        '''
        self.x = x
        self.u = u
        self.EA_pred = EA_pred

        # collocation points
        # x_collocation = torch.linspace( torch.min(x), torch.max(x) , 1000)
        # PDE residual loss
        du_dx =  EA_pred*torch.autograd.grad( outputs=u, inputs=[x], grad_outputs=torch.ones_like(u),create_graph=True, retain_graph=True )[0]
        f = torch.add(torch.autograd.grad( outputs=du_dx, inputs=[x], grad_outputs=torch.ones_like(du_dx),create_graph=True, retain_graph=True )[0], self.dist_load(x))
        differential_equation_loss = torch.mean( torch.square( f ))
        boundary_condition_loss = 0
        total_loss = differential_equation_loss + boundary_condition_loss
        return differential_equation_loss, total_loss
    
    def predict(self, x, u, model):
        """Get displacements."""
        '''
        This function is used while inference (you can even use in your training phase if needed.
        It takes x as input and returns model predicted displacement)
        Params:
            x - input spatial value
            u - model predicted displacement
        '''
        self.model.eval()
        EA_pred = self.model(torch.cat([x, u], 1))

        return EA_pred
    

    def train(self, epochs, optimizer, lr):
        """Train the model."""
        '''
        This function is used for training the network. While updating the model params use "costFunction" 
        function for calculating loss
        Params:
            epochs - number of epochs
            optimizer - name of the optimizer
            **kwarhs - additional params
        This function doesn't have any return values. Just print the losses during training
        '''
        
        self.optimizer = optimizer
        self.lr = lr
        self.epochs = epochs
        model = neural_network_model()

        if optimizer == "LBFGS":
            optimizer = torch.optim.LBFGS(model.parameters(), lr=lr, max_iter=50000, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09)
        else:
            optimizer = torch.optim.Adam(model.parameters())
        

        model.train()
        total_loss_list = []
        diff_loss_list = []
        epochs_list = []
        for e in range(1, epochs+1):

            optimizer.zero_grad()
            EA_pred = model(x,u)
            diff_loss, total_loss = self.costFunction(x, u, EA_pred, model)
            total_loss.backward(retain_graph=True)
            optimizer.step()
            
            if e % 500 == 0:
                total_loss_list.append(total_loss.item())
                diff_loss_list.append(diff_loss.item())
                epochs_list.append(e)
                print(f'Epoch {e+0:03}: | Total Loss: {total_loss.item():.8f} | Diff Loss: {diff_loss.item():.8f}')

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=total_loss_list, mode='lines', name='Total Loss'))
        # fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=BC_loss_list, mode='lines+markers', name='BC Loss'))
        # fig.add_trace(go.Scatter(x=list(range(1, epochs+1)), y=diff_loss_list, mode='markers', name='Diff Loss'))
        fig.update_layout(title='Loss vs Epochs', xaxis_title='Epochs', yaxis_title='Total Loss')
        fig.show()

        return model

In [22]:
class neural_network_model(nn.Module):
    def __init__(self):
        super(neural_network_model, self).__init__()

        self.layer1 = nn.Linear(2, 20)
        self.layer2 = nn.Linear(20, 20)
        self.layer_out = nn.Linear(20, 1)
        self.tanh = nn.Tanh()

    def forward(self, x,u):
        q = self.tanh(self.layer1(torch.cat([x, u], 1)))
        q = self.tanh(self.layer2(q))
        EA = self.layer_out(q)
        return EA

5. Next we train our model. The method/function 'train' is defined in the class PhysicsInformedBarModel

In [23]:
epochs = 10000
learningRate = 1e-3
pinnModel = PhysicsInformedBarModel(x, u, L, dist_load=distLoad) # use the designed custom class from physicsinformed.py file
print( "Total Loss and Diff Loss is SAME" )
model = pinnModel.train(epochs, optimizer='adam', lr=learningRate) # function defined in custom class for training

Total Loss and Diff Loss is SAME
Epoch 500: | Total Loss: 4.27970266 | Diff Loss: 4.27970266
Epoch 1000: | Total Loss: 3.34978557 | Diff Loss: 3.34978557
Epoch 1500: | Total Loss: 2.43221140 | Diff Loss: 2.43221140
Epoch 2000: | Total Loss: 0.86287391 | Diff Loss: 0.86287391
Epoch 2500: | Total Loss: 0.11304046 | Diff Loss: 0.11304046
Epoch 3000: | Total Loss: 0.05572495 | Diff Loss: 0.05572495
Epoch 3500: | Total Loss: 0.02990761 | Diff Loss: 0.02990761
Epoch 4000: | Total Loss: 0.01725971 | Diff Loss: 0.01725971
Epoch 4500: | Total Loss: 0.00880063 | Diff Loss: 0.00880063
Epoch 5000: | Total Loss: 0.00419123 | Diff Loss: 0.00419123
Epoch 5500: | Total Loss: 0.00203762 | Diff Loss: 0.00203762
Epoch 6000: | Total Loss: 0.00097220 | Diff Loss: 0.00097220
Epoch 6500: | Total Loss: 0.00048765 | Diff Loss: 0.00048765
Epoch 7000: | Total Loss: 0.00029714 | Diff Loss: 0.00029714
Epoch 7500: | Total Loss: 0.00026001 | Diff Loss: 0.00026001
Epoch 8000: | Total Loss: 0.00020005 | Diff Loss: 0.0

In [24]:
# training data 
model.eval()
EA_pred = model(x,u).detach().numpy().reshape(-1)
x = x.detach().numpy().reshape(-1)
u = u.detach().numpy().reshape(-1)
EA_anal = EA_analytic(x)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=EA_pred, mode='lines', name='NN Predicted Displacement'))
fig.add_trace(go.Scatter(x=x, y=EA_anal, mode='markers', name='Analytical Solution'))
fig.update_layout(title='Displacement vs EA Value | Training', xaxis_title='EA Value', yaxis_title='Displacement (u)')
fig.show()

6. We generate sample test data using utilities library and then predict the displacements at those test points

In [25]:
samples = 100
x_test = generate_grid_1d(L, samples)
u_test = u_analytic(x_test)

7. We plot predicted coefficient (EA) at test points

In [26]:
model.eval()
EA_pred = model(x_test,u_test).detach().numpy().reshape(-1)
x_test = x_test.detach().numpy().reshape(-1)
u_test = u_test.detach().numpy().reshape(-1)
EA_anal = EA_analytic(x_test)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test, y=EA_pred, mode='lines', name='NN Predicted Displacement'))
fig.add_trace(go.Scatter(x=x_test, y=EA_anal, mode='markers', name='Analytical Solution'))
fig.update_layout(title='Displacement vs EA Value | Testing', xaxis_title='EA Value', yaxis_title='Displacement (u)')
fig.show()